#create delta table

In [0]:
%sql 

create or replace table employee_demo
(
  emp_id int,
  emp_name string,
  gender string,
  sal int,
  dept string
)using delta 
 location '/FileStore/tables/delta/practice_delta'

In [0]:
emp_df=spark.read.format('delta').options(path='/FileStore/tables/delta/practice_delta').load()
display(emp_df)

emp_id,emp_name,gender,sal,dept


In [0]:
%sql 

INSERT INTO employee_demo VALUES (1, 'John Doe', 'Male', 5000, 'IT');
INSERT INTO employee_demo VALUES (2, 'Jane Smith', 'Female', 6000, 'HR');
INSERT INTO employee_demo VALUES (3, 'Michael Johnson', 'Male', 5500, 'Finance');
INSERT INTO employee_demo VALUES (4, 'Emily Davis', 'Female', 7000, 'Marketing');
INSERT INTO employee_demo VALUES (5, 'Chris Lee', 'Male', 4800, 'IT');
INSERT INTO employee_demo VALUES (6, 'Emma Wilson', 'Female', 6200, 'HR');
INSERT INTO employee_demo VALUES (7, 'David Brown', 'Male', 5300, 'Finance');

num_affected_rows,num_inserted_rows
1,1


In [0]:
emp_df.show()

+------+---------------+------+----+---------+
|emp_id|       emp_name|gender| sal|     dept|
+------+---------------+------+----+---------+
|     3|Michael Johnson|  Male|5500|  Finance|
|     4|    Emily Davis|Female|7000|Marketing|
|     7|    David Brown|  Male|5300|  Finance|
|     6|    Emma Wilson|Female|6200|       HR|
|     2|     Jane Smith|Female|6000|       HR|
|     5|      Chris Lee|  Male|4800|       IT|
|     1|       John Doe|  Male|5000|       IT|
+------+---------------+------+----+---------+



#Standard SQL using Table Name

In [0]:
%sql 

update employee_demo 
set sal=sal+500
where emp_id=6;


select * from employee_demo;

emp_id,emp_name,gender,sal,dept
3,Michael Johnson,Male,5500,Finance
4,Emily Davis,Female,7000,Marketing
7,David Brown,Male,5300,Finance
6,Emma Wilson,Female,7200,HR
2,Jane Smith,Female,6000,HR
5,Chris Lee,Male,4800,IT
1,John Doe,Male,5000,IT


#using table path in sql


In [0]:
%sql

update delta.`/FileStore/tables/delta/practice_delta`
set sal=5000 
where emp_id=6;

select * from employee_demo;

emp_id,emp_name,gender,sal,dept
3,Michael Johnson,Male,5500,Finance
4,Emily Davis,Female,7000,Marketing
7,David Brown,Male,5300,Finance
6,Emma Wilson,Female,5000,HR
2,Jane Smith,Female,6000,HR
5,Chris Lee,Male,4800,IT
1,John Doe,Male,5000,IT


#using tableInstance

In [0]:
from delta.tables import * 

from pyspark.sql.functions import *

In [0]:
emp_table_instance=DeltaTable.forName(spark,'employee_demo')

In [0]:
emp_table_instance.update(
                             condition="emp_id = 3",
                             set={'gender': "'F'"}
                         )

In [0]:
display(emp_table_instance.toDF())

emp_id,emp_name,gender,sal,dept
4,Emily Davis,Female,7000,Marketing
3,Michael Johnson,F,5500,Finance
7,David Brown,Male,5300,Finance
6,Emma Wilson,Female,5000,HR
2,Jane Smith,Female,6000,HR
5,Chris Lee,Male,4800,IT
1,John Doe,Male,5000,IT


In [0]:
emp_table_instance.update(
                             condition= col('emp_name')=='Emma Wilson',
                             set={'dept':lit('sales')}
                         )



In [0]:
%sql
select * from employee_demo;

emp_id,emp_name,gender,sal,dept
4,Emily Davis,Female,7000,Marketing
3,Michael Johnson,F,5500,Finance
7,David Brown,Male,5300,Finance
6,Emma Wilson,Female,5000,sales
2,Jane Smith,Female,6000,HR
5,Chris Lee,Male,4800,IT
1,John Doe,Male,5000,IT


In [0]:
emp_df=emp_table_instance.toDF()

In [0]:
emp_df=emp_df.withColumn('dept',when(col('dept')=='Finance','Economics').otherwise(col('dept')))

emp_df.show(7)

+------+---------------+------+----+---------+
|emp_id|       emp_name|gender| sal|     dept|
+------+---------------+------+----+---------+
|     4|    Emily Davis|Female|7000|Marketing|
|     3|Michael Johnson|     F|5500|Economics|
|     6|    Emma Wilson|Female|5000|    sales|
|     7|    David Brown|  Male|5300|Economics|
|     2|     Jane Smith|Female|6000|       HR|
|     5|      Chris Lee|  Male|4800|       IT|
|     1|       John Doe|  Male|5000|       IT|
+------+---------------+------+----+---------+

